In [215]:
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter

# 데이터를 읽어봅시다. 
train_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_train.txt')
test_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_test.txt')

train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [216]:
from konlpy.tag import Mecab

tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']


def load_data(train_data, test_data, num_words=10000):
    train_data.drop_duplicates(subset=['document'], inplace=True)
    train_data = train_data.dropna(how = 'any') 
    test_data.drop_duplicates(subset=['document'], inplace=True)
    test_data = test_data.dropna(how = 'any') 
    
    X_train = []
    for sentence in train_data['document']:
        temp_X = tokenizer.morphs(sentence) # 토큰화
        temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
        X_train.append(temp_X)

    X_test = []
    for sentence in test_data['document']:
        temp_X = tokenizer.morphs(sentence) # 토큰화
        temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
        X_test.append(temp_X)
    
    words = np.concatenate(X_train).tolist() # ?
    counter = Counter(words)
    counter = counter.most_common(10000 - 4)
    vocab = ['<PAD>', '<BOS>', '<UNK>', '<UNUSED>'] + [key for key, _ in counter]
    word_to_index = {word:index for index, word in enumerate(vocab)}
    word_to_index_cp = word_to_index
        
    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in wordlist]
        
    X_train = list(map(wordlist_to_indexlist, X_train))
    X_test = list(map(wordlist_to_indexlist, X_test))
        
    return X_train, np.array(list(train_data['label'])), X_test, np.array(list(test_data['label'])), word_to_index

In [217]:
X_train, y_train, X_test, y_test, word_to_index = load_data(train_data, test_data)

In [218]:
index_to_word = {index:word for word, index in word_to_index.items()}

In [219]:
# 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트 벡터로 변환해 주는 함수입니다. 
# 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다. 
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']]+[word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in sentence.split()]

# 여러 개의 문장 리스트를 한꺼번에 단어 인덱스 리스트 벡터로 encode해 주는 함수입니다. 
def get_encoded_sentences(sentences, word_to_index):
    return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

# 여러 개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
def get_decoded_sentences(encoded_sentences, index_to_word):
    return [get_decoded_sentence(encoded_sentence, index_to_word) for encoded_sentence in encoded_sentences]

In [220]:

# 테스트 데이터
total_data_text = list(X_train) + list(X_test)

# 텍스트데이터 문장길이의 리스트를 생성
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)

# 문장길이의 평균값, 최대값, 표준편차를 계산 
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 최대 길이(maxlen)를 (평균 + 2 * 표준편차)로 설정 
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)

문장길이 평균 :  15.96940191154864
문장길이 최대 :  116
문장길이 표준편차 :  12.843571191092


train과 test에 패딩을 추가하면서 나눴다. 

In [221]:

X_train = keras.preprocessing.sequence.pad_sequences(X_train,
                                                        value=word_to_index['<PAD>'],
                                                        padding='pre', # 혹은 'pre'
                                                        maxlen=maxlen)

X_test = keras.preprocessing.sequence.pad_sequences(X_test,
                                                       value=word_to_index['<PAD>'],
                                                       padding='pre', # 혹은 'pre'
                                                       maxlen=maxlen)

In [222]:
# 약 8:2 비율로 
# validation set 30000건 분리
X_val = X_train[:30000]   
y_val = y_train[:30000]

# validation set을 제외한 나머지 
train_x = X_train[30000:]  
train_y = y_train[30000:]


In [ ]:
학습방법 3가지의 정확성을 확인해보았다. 

1-D Convolution Neural Network(1-D CNN)

In [223]:
vocab_size = 10000 # 어휘 사전의 크기입니다(10개의 단어)
word_vector_dim = 16   # 단어 하나를 표현하는 임베딩 벡터의 차원 수입니다. 

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(keras.layers.Conv1D(16, 7, activation='relu'))
model.add(keras.layers.MaxPooling1D(5))
model.add(keras.layers.Conv1D(16, 7, activation='relu'))
model.add(keras.layers.GlobalMaxPooling1D())
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model.summary()

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_33 (Embedding)     (None, None, 16)          160000    
_________________________________________________________________
conv1d_22 (Conv1D)           (None, None, 16)          1808      
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, None, 16)          0         
_________________________________________________________________
conv1d_23 (Conv1D)           (None, None, 16)          1808      
_________________________________________________________________
global_max_pooling1d_19 (Glo (None, 16)                0         
_________________________________________________________________
dense_54 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_55 (Dense)             (None, 1)               

In [224]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs = 10  

history = model.fit(train_x,
                    train_y,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(X_val, y_val),
                    verbose=1)

Epoch 1/10
227/227 [==============================] - 3s 8ms/step - loss: 0.6094 - accuracy: 0.6526 - val_loss: 0.3559 - val_accuracy: 0.8443
Epoch 2/10
227/227 [==============================] - 2s 9ms/step - loss: 0.3309 - accuracy: 0.8593 - val_loss: 0.3406 - val_accuracy: 0.8529
Epoch 3/10
227/227 [==============================] - 2s 8ms/step - loss: 0.2862 - accuracy: 0.8819 - val_loss: 0.3403 - val_accuracy: 0.8540
Epoch 4/10
227/227 [==============================] - 2s 8ms/step - loss: 0.2577 - accuracy: 0.8954 - val_loss: 0.3492 - val_accuracy: 0.8523
Epoch 5/10
227/227 [==============================] - 2s 8ms/step - loss: 0.2217 - accuracy: 0.9157 - val_loss: 0.3727 - val_accuracy: 0.8470
Epoch 6/10
227/227 [==============================] - 2s 8ms/step - loss: 0.1867 - accuracy: 0.9315 - val_loss: 0.3982 - val_accuracy: 0.8433
Epoch 7/10
227/227 [==============================] - 2s 8ms/step - loss: 0.1553 - accuracy: 0.9466 - val_loss: 0.4325 - val_accuracy: 0.8390
Epoch 

In [225]:
results = model.evaluate(X_test,  y_test, verbose=2)

print(results)

1537/1537 - 2s - loss: 0.5725 - accuracy: 0.8264
[0.5724867582321167, 0.8263930082321167]


GlobalMaxPooling1D

In [226]:

vocab_size = 10000    # 어휘 사전의 크기입니다(10,000개의 단어)
word_vector_dim = 16  # 워드 벡터의 차원 수 (변경 가능한 하이퍼파라미터)

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(keras.layers.GlobalMaxPooling1D())
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model.summary()

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_34 (Embedding)     (None, None, 16)          160000    
_________________________________________________________________
global_max_pooling1d_20 (Glo (None, 16)                0         
_________________________________________________________________
dense_56 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_57 (Dense)             (None, 1)                 9         
Total params: 160,145
Trainable params: 160,145
Non-trainable params: 0
_________________________________________________________________


In [227]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs = 10  

history = model.fit(train_x,
                    train_y,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(X_val, y_val),
                    verbose=1)

Epoch 1/10
227/227 [==============================] - 2s 5ms/step - loss: 0.6558 - accuracy: 0.6295 - val_loss: 0.4431 - val_accuracy: 0.8185
Epoch 2/10
227/227 [==============================] - 1s 5ms/step - loss: 0.4044 - accuracy: 0.8343 - val_loss: 0.3651 - val_accuracy: 0.8400
Epoch 3/10
227/227 [==============================] - 1s 4ms/step - loss: 0.3337 - accuracy: 0.8605 - val_loss: 0.3522 - val_accuracy: 0.8455
Epoch 4/10
227/227 [==============================] - 1s 5ms/step - loss: 0.3000 - accuracy: 0.8764 - val_loss: 0.3514 - val_accuracy: 0.8485
Epoch 5/10
227/227 [==============================] - 1s 4ms/step - loss: 0.2765 - accuracy: 0.8882 - val_loss: 0.3557 - val_accuracy: 0.8478
Epoch 6/10
227/227 [==============================] - 1s 4ms/step - loss: 0.2560 - accuracy: 0.8976 - val_loss: 0.3622 - val_accuracy: 0.8472
Epoch 7/10
227/227 [==============================] - 1s 5ms/step - loss: 0.2435 - accuracy: 0.9026 - val_loss: 0.3716 - val_accuracy: 0.8464
Epoch 

In [228]:
results = model.evaluate(X_test,  y_test, verbose=2)

print(results)

1537/1537 - 2s - loss: 0.4167 - accuracy: 0.8353
[0.416698157787323, 0.8353235721588135]


LSTM

In [229]:
vocab_size = 10000  # 어휘 사전의 크기입니다(10000개의 단어)
word_vector_dim = 20  # 단어 하나를 표현하는 임베딩 벡터의 차원수입니다. 

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(keras.layers.LSTM(8))   # 가장 널리 쓰이는 RNN인 LSTM 레이어를 사용하였습니다. 이때 LSTM state 벡터의 차원수는 8로 하였습니다. (변경 가능)
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model.summary()

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_35 (Embedding)     (None, None, 20)          200000    
_________________________________________________________________
lstm_8 (LSTM)                (None, 8)                 928       
_________________________________________________________________
dense_58 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_59 (Dense)             (None, 1)                 9         
Total params: 201,009
Trainable params: 201,009
Non-trainable params: 0
_________________________________________________________________


In [230]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs = 10  

history = model.fit(train_x,
                    train_y,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(X_val, y_val),
                    verbose=1)

Epoch 1/10
227/227 [==============================] - 4s 12ms/step - loss: 0.6137 - accuracy: 0.6786 - val_loss: 0.3763 - val_accuracy: 0.8413
Epoch 2/10
227/227 [==============================] - 2s 11ms/step - loss: 0.3538 - accuracy: 0.8527 - val_loss: 0.3488 - val_accuracy: 0.8485
Epoch 3/10
227/227 [==============================] - 2s 11ms/step - loss: 0.3181 - accuracy: 0.8682 - val_loss: 0.3463 - val_accuracy: 0.8493
Epoch 4/10
227/227 [==============================] - 2s 11ms/step - loss: 0.2989 - accuracy: 0.8760 - val_loss: 0.3451 - val_accuracy: 0.8511
Epoch 5/10
227/227 [==============================] - 2s 11ms/step - loss: 0.2856 - accuracy: 0.8814 - val_loss: 0.3460 - val_accuracy: 0.8501
Epoch 6/10
227/227 [==============================] - 2s 10ms/step - loss: 0.2759 - accuracy: 0.8857 - val_loss: 0.3482 - val_accuracy: 0.8497
Epoch 7/10
227/227 [==============================] - 2s 11ms/step - loss: 0.2616 - accuracy: 0.8918 - val_loss: 0.3554 - val_accuracy: 0.8487

In [231]:
results = model.evaluate(X_test,  y_test, verbose=2)

print(results)

1537/1537 - 5s - loss: 0.4138 - accuracy: 0.8425
[0.41378098726272583, 0.8424842953681946]


In [ ]:
word2vec을 사용해서 다시 추가하고 모델 3개를 다시 돌려보았다. 

In [232]:
# word2vec 사용 

from gensim.models.keyedvectors import Word2VecKeyedVectors
from tensorflow.keras.initializers import Constant
import gensim

word2vec_path =  os.getenv('HOME')+"/aiffel/sentiment_classification/ko.bin"
f = open(word2vec_path, 'w')
f.write('{} {}\n'.format(vocab_size-4, word_vector_dim)) 
# 단어 개수(에서 특수문자 4개는 제외하고)만큼의 워드 벡터를 파일에 기록
vectors = model.get_weights()[0]
for i in range(4,vocab_size):
    f.write('{} {}\n'.format(index_to_word[i], ' '.join(map(str, list(vectors[i, :])))))
f.close()

word_vectors = Word2VecKeyedVectors.load_word2vec_format(word2vec_file_path, binary=False)

embedding_matrix = np.random.rand(vocab_size, word_vector_dim)

# embedding_matrix에 Word2Vec 워드 벡터를 단어 하나씩마다 차례차례 카피
for i in range(4,vocab_size):
    if index_to_word[i] in word_vectors:
        embedding_matrix[i] = word_vectors[index_to_word[i]]

In [233]:
word_vectors.similar_by_word("재미")

[('너무', 0.9164605140686035),
 ('넘', 0.8898469805717468),
 ('이제훈', 0.8803137540817261),
 ('야지', 0.8700798153877258),
 ('씌', 0.8554890155792236),
 ('드러나', 0.8503690958023071),
 ('자랑', 0.841502845287323),
 ('임청하', 0.8413569331169128),
 ('철저', 0.8336808681488037),
 ('리그', 0.8323181867599487)]

1-D Convolution Neural Network(1-D CNN)

In [234]:
#
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 
                                 word_vector_dim, 
                                 embeddings_initializer=Constant(embedding_matrix),  # 카피한 임베딩을 여기서 활용
                                 input_length=maxlen, 
                                 trainable=True))   # trainable을 True로 주면 Fine-tuning
model.add(keras.layers.Conv1D(16, 7, activation='relu'))
model.add(keras.layers.MaxPooling1D(5))
model.add(keras.layers.Conv1D(16, 7, activation='relu'))
model.add(keras.layers.GlobalMaxPooling1D())
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid')) 

model.summary()

Model: "sequential_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_36 (Embedding)     (None, 41, 20)            200000    
_________________________________________________________________
conv1d_24 (Conv1D)           (None, 35, 16)            2256      
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 7, 16)             0         
_________________________________________________________________
conv1d_25 (Conv1D)           (None, 1, 16)             1808      
_________________________________________________________________
global_max_pooling1d_21 (Glo (None, 16)                0         
_________________________________________________________________
dense_60 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_61 (Dense)             (None, 1)               

In [235]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs = 10  

history = model.fit(train_x,
                    train_y,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(X_val, y_val),
                    verbose=1)

Epoch 1/10
227/227 [==============================] - 3s 9ms/step - loss: 0.5364 - accuracy: 0.7257 - val_loss: 0.3564 - val_accuracy: 0.8463
Epoch 2/10
227/227 [==============================] - 2s 8ms/step - loss: 0.2935 - accuracy: 0.8806 - val_loss: 0.3493 - val_accuracy: 0.8501
Epoch 3/10
227/227 [==============================] - 2s 8ms/step - loss: 0.2741 - accuracy: 0.8899 - val_loss: 0.3535 - val_accuracy: 0.8515
Epoch 4/10
227/227 [==============================] - 2s 8ms/step - loss: 0.2582 - accuracy: 0.8987 - val_loss: 0.3536 - val_accuracy: 0.8515
Epoch 5/10
227/227 [==============================] - 2s 8ms/step - loss: 0.2410 - accuracy: 0.9070 - val_loss: 0.3632 - val_accuracy: 0.8517
Epoch 6/10
227/227 [==============================] - 2s 8ms/step - loss: 0.2212 - accuracy: 0.9156 - val_loss: 0.3765 - val_accuracy: 0.8491
Epoch 7/10
227/227 [==============================] - 2s 8ms/step - loss: 0.1955 - accuracy: 0.9278 - val_loss: 0.3907 - val_accuracy: 0.8500
Epoch 

In [236]:
results = model.evaluate(X_test,  y_test, verbose=2)

print(results)

1537/1537 - 2s - loss: 0.5025 - accuracy: 0.8350
[0.5024564266204834, 0.8349777460098267]


GlobalMaxPooling1D

In [237]:

vocab_size = 10000    # 어휘 사전의 크기입니다(10,000개의 단어)
word_vector_dim = 16  # 워드 벡터의 차원 수 (변경 가능한 하이퍼파라미터)

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(keras.layers.GlobalMaxPooling1D())
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model.summary()

Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_37 (Embedding)     (None, None, 16)          160000    
_________________________________________________________________
global_max_pooling1d_22 (Glo (None, 16)                0         
_________________________________________________________________
dense_62 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_63 (Dense)             (None, 1)                 9         
Total params: 160,145
Trainable params: 160,145
Non-trainable params: 0
_________________________________________________________________


In [238]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs = 10  

history = model.fit(train_x,
                    train_y,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(X_val, y_val),
                    verbose=1)

Epoch 1/10
227/227 [==============================] - 2s 5ms/step - loss: 0.6554 - accuracy: 0.6547 - val_loss: 0.4405 - val_accuracy: 0.8211
Epoch 2/10
227/227 [==============================] - 1s 4ms/step - loss: 0.4013 - accuracy: 0.8364 - val_loss: 0.3602 - val_accuracy: 0.8425
Epoch 3/10
227/227 [==============================] - 1s 5ms/step - loss: 0.3280 - accuracy: 0.8624 - val_loss: 0.3486 - val_accuracy: 0.8482
Epoch 4/10
227/227 [==============================] - 1s 4ms/step - loss: 0.2969 - accuracy: 0.8770 - val_loss: 0.3486 - val_accuracy: 0.8488
Epoch 5/10
227/227 [==============================] - 1s 4ms/step - loss: 0.2765 - accuracy: 0.8863 - val_loss: 0.3523 - val_accuracy: 0.8500
Epoch 6/10
227/227 [==============================] - 1s 5ms/step - loss: 0.2554 - accuracy: 0.8983 - val_loss: 0.3587 - val_accuracy: 0.8486
Epoch 7/10
227/227 [==============================] - 1s 4ms/step - loss: 0.2423 - accuracy: 0.9051 - val_loss: 0.3684 - val_accuracy: 0.8468
Epoch 

In [239]:
results = model.evaluate(X_test,  y_test, verbose=2)

print(results)

1537/1537 - 2s - loss: 0.4193 - accuracy: 0.8378
[0.4192637801170349, 0.8378461003303528]


LSTM

In [261]:
vocab_size = 10000  # 어휘 사전의 크기입니다(10000개의 단어)
word_vector_dim = 200  # 단어 하나를 표현하는 임베딩 벡터의 차원수입니다. 

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(keras.layers.LSTM(8))   # 가장 널리 쓰이는 RNN인 LSTM 레이어를 사용하였습니다. 이때 LSTM state 벡터의 차원수는 8로 하였습니다. (변경 가능)
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model.summary()

Model: "sequential_50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_49 (Embedding)     (None, None, 200)         2000000   
_________________________________________________________________
lstm_14 (LSTM)               (None, 8)                 6688      
_________________________________________________________________
dense_74 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_75 (Dense)             (None, 1)                 9         
Total params: 2,006,769
Trainable params: 2,006,769
Non-trainable params: 0
_________________________________________________________________


In [266]:
model.compile(optimizer='RMSprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs =10

history = model.fit(train_x,
                    train_y,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(X_val, y_val),
                    verbose=1)

Epoch 1/10
227/227 [==============================] - 8s 27ms/step - loss: 0.0591 - accuracy: 0.9807 - val_loss: 0.8041 - val_accuracy: 0.8303
Epoch 2/10
227/227 [==============================] - 5s 23ms/step - loss: 0.0524 - accuracy: 0.9830 - val_loss: 0.8173 - val_accuracy: 0.8301
Epoch 3/10
227/227 [==============================] - 5s 22ms/step - loss: 0.0510 - accuracy: 0.9835 - val_loss: 0.8502 - val_accuracy: 0.8294
Epoch 4/10
227/227 [==============================] - 5s 22ms/step - loss: 0.0512 - accuracy: 0.9827 - val_loss: 0.8514 - val_accuracy: 0.8286
Epoch 5/10
227/227 [==============================] - 5s 22ms/step - loss: 0.0518 - accuracy: 0.9827 - val_loss: 0.8346 - val_accuracy: 0.8268
Epoch 6/10
227/227 [==============================] - 5s 22ms/step - loss: 0.0508 - accuracy: 0.9830 - val_loss: 0.8594 - val_accuracy: 0.8281
Epoch 7/10
227/227 [==============================] - 5s 22ms/step - loss: 0.0481 - accuracy: 0.9840 - val_loss: 0.8258 - val_accuracy: 0.8290

In [267]:
results = model.evaluate(X_test,  y_test, verbose=2)

print(results)

1537/1537 - 4s - loss: 0.8437 - accuracy: 0.8256
[0.8436509370803833, 0.8255792856216431]


#회고 
word2vec을 사용하는데 너무 많은 시간을 썼는데 생각보다 정확도가 많이 늘지 않아서 실망했다. 파리미터를 튜닝하면서 해나갔어야됐는데 word2vec을 사용하는데 시간을 너무 많이 써버려서 하지 못했다. 다음엔 학습 방법에 대해 더 많이 신경써야겠다. 